# Prepare input parameters

In [ ]:
from bokeh.plotting import output_notebook
from bokeh.plotting import show

import geopandas as gpd

from shapely.geometry import Point
from shapely.ops import linemerge

from easy_map_bokeh import EasyMapBokeh

from osmgt import OsmGt

from graph_tool.topology import shortest_path


output_notebook()


location = "Roanne"


# Get POIs

In [ ]:
%%time

poi_from_location = OsmGt.poi_from_location(location)
poi_from_location_gdf = poi_from_location.get_gdf()

# Get Roads

In [ ]:
%%time
roads_from_location = OsmGt.roads_from_location(location, mode="vehicle", additionnal_nodes=poi_from_location_gdf)
roads_from_location_gdf = roads_from_location.get_gdf()

# Plot Data

In [ ]:
# apply colors based on topology field
roads_from_location_gdf = roads_from_location_gdf[["topo_uuid", "id", "topology", "geometry"]]

roads_gdf_unchanged = roads_from_location_gdf.loc[roads_from_location_gdf["topology"] == "unchanged"]
roads_gdf_added = roads_from_location_gdf.loc[roads_from_location_gdf["topology"] == "added"]
roads_gdf_split = roads_from_location_gdf.loc[roads_from_location_gdf["topology"] == "split"]

poi_from_location_gdf = poi_from_location_gdf[["topo_uuid", "id", "geometry"]]


# node added on the graph
columns_without_geometry = roads_gdf_added.columns.tolist()
columns_without_geometry.remove("geometry")
node_added_on_the_graph = roads_gdf_added.copy(deep=True)
node_added_on_the_graph["geometry"] = node_added_on_the_graph["geometry"].apply(lambda x: Point(x.coords[-1]))
node_added_on_the_graph = node_added_on_the_graph[["topo_uuid", "id", "geometry"]]

#line split segments limits
columns_without_geometry = roads_gdf_split.columns.tolist()
columns_without_geometry.remove("geometry")
line_split_segments_limits = roads_gdf_split.copy(deep=True)
line_split_segments_limits["geometry"] = line_split_segments_limits["geometry"].apply(lambda x: [[Point(f) for f in x.coords][0], [Point(f) for f in x.coords][-1]])
line_split_segments_limits.set_index(columns_without_geometry, inplace=True)
line_split_segments_limits = line_split_segments_limits["geometry"].explode().reset_index()
line_split_segments_limits = line_split_segments_limits[["topo_uuid", "id", "geometry"]]
line_split_segments_limits = gpd.GeoDataFrame(line_split_segments_limits, geometry='geometry', crs="EPSG:4326")


layers_to_add = [
    {
        "input_gdf": roads_gdf_unchanged,
        "legend": "roads unchanged",
        "color": "black",
    },
    {
        "input_gdf": roads_gdf_added,
        "legend": "roads added",
        "color": "green",
    },
    {
        "input_gdf": roads_gdf_split,
        "legend": "roads split",
        "color": "orange",
    },
    {
        "input_gdf": poi_from_location_gdf,
        "legend": "POIs",
        "style": "square",
        "fill_color": "blue",
        "size": 9
    },
    {
        "input_gdf": line_split_segments_limits,
        "legend": "roads split nodes",
        "fill_color": "brown",
        "style": "circle",
        "size": 9
    },
    {
        "input_gdf": node_added_on_the_graph,
        "legend": "Nodes added",
        "fill_color": "red",
    },

]

my_map = EasyMapBokeh(
    "My roads and POIs",
    layers=layers_to_add
)
show(my_map.figure)

# Get network and display it

In [ ]:
%%time
graph = roads_from_location.get_graph()

graph.plot()


# Try a shortest path and display it

In [ ]:
%%time
# now, we have to define a start point and a end point and get their wkt
start_node_topo_uuid = 47
end_node_topo_uuid = 63

# 'topo_uuid' is generated by osmgt (during the topology processing).
# Some roads has been split that's whyso this id has been created.
start_node_wkt = poi_from_location_gdf[poi_from_location_gdf['topo_uuid'] == start_node_topo_uuid].iloc[0]["geometry"].wkt
end_node_wkt = poi_from_location_gdf[poi_from_location_gdf['topo_uuid'] == end_node_topo_uuid].iloc[0]["geometry"].wkt

# the graph have some methods (graph-tools method always exists!) to find egdes, vertices... Let's use the .find_vertex_from_name(). the wkt is the vertex name...
source_vertex = graph.find_vertex_from_name(start_node_wkt)
target_vertex = graph.find_vertex_from_name(end_node_wkt)

# shortest path computing...
path_vertices, path_edges = shortest_path(
    graph,
    source=source_vertex,
    target=target_vertex,
    weights=graph.edge_weights  # weigth is based on line length
)

# get path by using edge names
roads_ids = [
    graph.edge_names[edge]
    for edge in path_edges
]

roads_gdf_copy = roads_from_location_gdf.copy(deep=True)
shortest_path_found = roads_gdf_copy[roads_from_location_gdf['topo_uuid'].isin(roads_ids)].to_crs(3857)['geometry'].to_list()
shortest_path_found_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:3857", geometry=[linemerge(shortest_path_found)])

layers_to_add = [
    {
        "input_gdf": shortest_path_found_gdf,
        "legend": "shortest_path",
        "color": "red",
        "line_width": 5
    },
]

my_shortest_path_map = EasyMapBokeh(
    "My shortest path",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)